In [1]:
import datetime
import praw
import pandas as pd
import numpy as np
import re
from keys import client_id, client_secret, username

## 1. Importing Netflix shows dataset (Kaggle)

Due to there being no official Netflix API for me to gather data on Netflix shows and their respective genre, I have instead decided to use this dataset I found on Kaggle, which consists of TV shows and movies available on Netflix as of 2019. The dataset was collected from Flixable which is a third-party Netflix search engine.

Link: https://www.kaggle.com/shivamb/netflix-shows

In [2]:
netflix_shows = pd.read_csv("netflix_titles.csv")

In [ ]:
netflix_shows.head()

In [3]:
netflix_shows.shape

(6234, 12)

Dropping irrelevant columns (for my aims)

In [4]:
# Dropping columns
netflix_shows = netflix_shows.drop(columns = ["director", "cast", "rating", "duration", "description"])

In [5]:
netflix_shows.shape

(6234, 7)

I am also going to remove some titles which are also commonly used words, as well as titles I know are outliers, e.g. removing "Philadelphia" as I know the matches are false matches with "It's Always Sunny in Philadelphia" - which is not in the Netflix dataset.

Thankfully, all these titles are quite unpopular! (Appears very little in top posts results / not at all)

In [5]:
remove_words = ["Philadelphia", "Next", "After", "Sometimes", "Game", "Episodes", "Life", "Security", "Victor", "Money",
                "Together", "Rotten", "Close", "Tiger", "Justice", "Last", "Wanted"]
remove_words_index = []

In [6]:
for word in remove_words:
    remove_words_index.append(netflix_shows.loc[netflix_shows["title"] == word].index.tolist())

In [ ]:
# List comprehension to "flatten" list
remove_words_index = [x for y in remove_words_index for x in y]
remove_words_index

In [8]:
# Dropping rows
netflix_shows = netflix_shows.drop(remove_words_index, axis = 0)

In [9]:
netflix_shows.shape

(6215, 7)

I am now going to sort by length for the "title" column, descending.

And also resetting the index.

In [10]:
netflix_shows.index = netflix_shows["title"].str.len()
netflix_shows = netflix_shows.sort_index(ascending = False)
netflix_shows = netflix_shows.reset_index(drop = True)

In [ ]:
netflix_shows

Dropping rows in which the len(title) is <= 3 characters. This is because short phrases can easily be false matches later on when I am matching the Netflix title to the Reddit Submission.

Unfortunately as a result, these titles won't be considered in my sentiment analysis. (Interestingly, they are also not very popular!)

In [ ]:
# Netflix titles with 3 or less characters in length start at index 6168
netflix_shows.loc[netflix_shows["title"].str.len() <= 3]

In [12]:
# Only keeping rows up to index 6167 (dropping rows 6168 and after)
netflix_shows = netflix_shows.loc[:6167]

In [ ]:
netflix_shows

In [13]:
# Writing to a new csv
netflix_shows.to_csv("netflix_titles_v2.csv", index = False)

## 2. Collecting the Posts from r/Netflix

In [17]:
netflix_shows = pd.read_csv("netflix_titles_v2.csv")

Initialising a Reddit instance.

In [14]:
reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent=f"android:my_app:v1 (by /u/{username})")

Calling the Reddit API and building a dataframe from it.

In [15]:
netflix = reddit.subreddit("netflix")

# Get the top 1000 posts, with title, url, body, upvotes, timestamp, and an index that serves as a key between
# the posts and the comments we collect later
posts = []
for index, post in enumerate(netflix.top(limit = 1000)):
    posts.append([post.title, post.selftext, "https://www.reddit.com" + post.permalink, post.score, post.created_utc, index])

# Converting into DataFrame
posts = pd.DataFrame(posts, columns = ["Submission", "Body", "URL", "Upvotes", "Time", "Key"])

# Changing from UTC time to standard timestamp
posts.Time = posts.Time.apply(lambda x: pd.to_datetime(datetime.datetime.fromtimestamp(x)))

In [ ]:
posts.head()

In [16]:
# Looks like we have only gathered 988 posts! (out of a total of ~118000 total submissions)
posts.shape

(988, 6)

Now, I am looking for specifically posts which are discussions based around a TV show / movie. So first filter out posts which don't contain a TV show / movie title, and keep the posts which do.

In [18]:
# List of unique Netflix titles
titles = netflix_shows["title"]
titles.head()

0    Jim & Andy: The Great Beyond - Featuring a Ver...
1    The Power of Grayskull: The Definitive History...
2    Mike Birbiglia: What I Should Have Said Was No...
3    Steve Martin and Martin Short: An Evening You ...
4    Cultivating the Seas: History and Future of th...
Name: title, dtype: object

Here I initially tried regex, but it didn't work out, which is fine.

In [19]:
# Substrings have special characters such as $ and ^ which I wanted to match literally. These characters have specific
# meanings in the context of regular expressions and will affect the matching. So, I had to make my list of
# substrings (titles) "safer" by escaping non-alphanumeric characters with re.escape.

In [20]:
# titles_safe = [re.escape(t) for t in titles]
# titles_safe[0:5]

In [21]:
# Using the regex | character to try to match each of the substrings in the Submission title.
# posts = posts.loc[posts["Submission"].str.contains("|".join(titles_safe))]

Using a for loop to match the corresponding Netflix title for each Reddit post (later adding 2 columns "Title" and "Genres" as well). Since I sorted my Netflix titles dataset from longest title to shortest, the following loop will match the longer titles before matching a shorter one, which is what I want.

In [22]:
# Creating empty column for the corresponding Netflix title
posts.insert(1, "Netflix_Title", "")

# Creating empty column for the corresponding Genres (listed in)
posts.insert(2, "Genres", "")

In [ ]:
# Using a for loop to match the corresponding Netflix title for each Reddit post

# Storing the index of matching rows here
netflix_index = []

# Storing the matching titles here
netflix_match_titles = []

# Storing the corresponding genres here
netflix_genres = []

count = 0
for line in posts["Submission"]:
    for title in titles:
        if title in line:
            count += 1
            netflix_index.append(posts.index[posts["Submission"] == line].tolist())
            netflix_match_titles.append(title)
            netflix_genres.append(netflix_shows[netflix_shows["title"] == title].listed_in.tolist())
            print(str(count) + ": " + title + " - index " + str(posts.index[posts["Submission"] == line].tolist()))
            print(line + "\n")
            break
        else:
            pass

In [24]:
# List comprehension to "flatten" netflix_index
netflix_index = [x for y in netflix_index for x in y]

In [25]:
# There are actually duplicates for 2 index values - at indices 205 and 247, removing them now
netflix_index.pop(205)
netflix_index.pop(247)

522

In [26]:
# Keeping only rows that match the index created in the for loop above
posts = posts.loc[netflix_index]

In [55]:
# Adding the matching titles to the "Netflix_Title" column
posts["Netflix_Title"] = netflix_match_titles

In [28]:
# Adding the matching genres to the Genres column
posts["Genres"] = netflix_genres

# Stripping the square brackets and single speech marks
posts["Genres"] = posts["Genres"].str.strip("[]'")

In [29]:
# Resetting the index and Key columns
posts.reset_index(drop = True, inplace = True)
posts = posts.drop(columns = ["Key"])
posts["Key"] = posts.index

In [101]:
posts

,Submission,Netflix_Title,Genres,Body,URL,Upvotes,Time,Key
0,Netflix executives say the success of 'Bright'...,Bright,"Action & Adventure, Sci-Fi & Fantasy",NaN,https://www.reddit.com/r/netflix/comments/7t2f...,6802,2018-01-26 18:11:49,0
1,Netflix Fires Kevin Spacey from ‘House of Cards’,House of Cards,"TV Dramas, TV Thrillers",NaN,https://www.reddit.com/r/netflix/comments/7ao8...,6504,2017-11-04 15:47:43,1
2,Netflix has Cloverfield 3,Cloverfield,"Action & Adventure, Horror Movies, Sci-Fi & Fa...",Edit: Streaming starts post game,https://www.reddit.com/r/netflix/comments/7vav...,5597,2018-02-05 13:23:43,2
3,The Witcher is Netflix's highest rated series,The Witcher,"TV Action & Adventure, TV Dramas, TV Mysteries",https://www.forbes.com/sites/paultassi/2019/12...,https://www.reddit.com/r/netflix/comments/eekm...,4891,2019-12-24 02:30:46,3
4,"Netflix has won its first Oscar, for Icarus",Icarus,"Documentaries, Sports Movies",NaN,https://www.reddit.com/r/netflix/comments/8238...,4632,2018-03-05 17:03:47,4
...,...,...,...,...,...,...,...,...
410,Master of None season 2 premieres May 12 [ALL],Master of None,TV Comedies,NaN,https://www.reddit.com/r/netflix/comments/5zkl...,631,2017-03-16 06:03:06,410
411,BoJack Horseman | Season 4 Official Trailer [H...,BoJack Horseman,TV Comedies,NaN,https://www.reddit.com/r/netflix/comments/6vri...,627,2017-08-25 03:15:41,411
412,House of Cards final season is coming November 2,House of Cards,"TV Dramas, TV Thrillers",NaN,https://www.reddit.com/r/netflix/comments/95c0...,624,2018-08-08 02:15:57,412
413,Black Mirror: Striking Vipers | Official Trailer,Black Mirror,"British TV Shows, International TV Shows, TV D...",NaN,https://www.reddit.com/r/netflix/comments/brbc...,622,2019-05-22 02:52:45,413


In [50]:
posts.shape

(415, 8)

#### Looking good!

We now have roughly 40% (~415) of the top 988 posts on r/Netflix, that are discussions based around a TV show / movie.

We have successfully attached each to each post their respective Netflix title and genres listed.

In [103]:
# Writing the posts to a csv
posts.to_csv("Netflix_Posts.csv", index = False)

## 3. Collecting the comments from each post

In [98]:
# Loading in relevant files
netflix_shows = pd.read_csv("netflix_titles_v2.csv")
posts = pd.read_csv("Netflix_Posts.csv")

Function to collect every comment.

In [34]:
def collect_replies(key, url):
    ''' 
    params pandas series row: each row of the dataframe we built above in the form of a panda series
    Returns a pandas DataFrame, where each row represents an individual comment
    '''
    submission = reddit.submission(url = url)
    submission.comments.replace_more(limit = None)
    comment_queue = submission.comments[:]
    
    table = {"Reply":[], "Upvote":[], "Time":[], "Key":[]}
    
    while comment_queue:
        comment = comment_queue.pop(0)
        table["Reply"].append(comment.body)
        table["Time"].append(comment.created_utc)
        table["Upvote"].append(comment.score)
        table["Key"].append(key)
        comment_queue.extend(comment.replies)
    
    return pd.DataFrame.from_dict(table)

Creating dataframe of comments. Using list comprehensions will speed things up slightly.

In [39]:
# Generate a list of tuples that contains the Key and URL for each row
# First value of tuple is Key, second is URL
keys = posts.Key.tolist()
urls = posts.URL.tolist()
tuples = list(zip(keys, urls))

# Generate our comments dataframe using list comprehensions
comments = pd.concat([collect_replies(x[0], x[1]) for x in tuples])

Took me ~15 minutes to generate the comments for 415 posts!

In [40]:
# Again, converting the timsteamp from UTC to a standard format
comments.Time = comments.Time.apply(lambda x: pd.to_datetime(datetime.datetime.fromtimestamp(x)))

In [43]:
comments.head()

,Reply,Upvote,Time,Key
0,I didn't go into Bright expecting a cinematic ...,2079,2018-01-26 21:23:11,0
1,It was pretty good. It was a solid B. Pretty m...,1529,2018-01-26 18:59:11,0
2,"I enjoyed it, I would watch another. It did n...",244,2018-01-27 00:16:08,0
3,This is a pretty terrible argument. There no f...,25,2018-01-27 04:42:08,0
4,Everything was great about it except the plot....,149,2018-01-26 23:37:54,0


In [44]:
comments.shape

(57732, 4)

We have gathered 57732 comments and replies from 415 posts!

In [53]:
comments.to_csv("Netflix_Comments.csv", index = False)